## Test Parser

In [1]:
from Parser import parse_LLM_output
from nltk.sem.logic import *
import re

In [10]:
# Possible Outputs from LM:
nothing = ""
anything = "sdaspdjasd"
partially_correct = "World model: ."
partially_correct_missing_info = "World model: ."
another_partially_correct = "Keys: ."
another_partially_correct_missing_info = "Keys: ."
empty_answer = "World model: . Keys: "
syntactically_correct_thing = "World model: Rey is Zany. Louie is Zany. Keys: F: Zany. p: Rey. r: Louie.."
no_model_no_keys_but_correct = "Rey is Zany. Louie is Zany. F: Zany. p: Rey. r: Louie.."

In [3]:
parse_LLM_output(nothing)

[]

In [4]:
parse_LLM_output(anything)

[]

In [5]:
parse_LLM_output(partially_correct)

kv: 
['World model', '']


[]

In [6]:
parse_LLM_output(partially_correct_missing_info)

kv: 
['World model', '']


[]

In [7]:
parse_LLM_output(another_partially_correct)

kv: 
['Keys', '']


[]

In [8]:
parse_LLM_output(another_partially_correct_missing_info)

kv: 
['Keys', '']


[]

In [9]:
parse_LLM_output(empty_answer)

. 
['']
kv: 
['']


[]

In [10]:
parse_LLM_output(syntactically_correct_thing)

0:World model: Rey is Zany. Louie is Zany. 
1:: F: Zany. p: Rey. r: Louie..
kv: 
['F', 'Zany']
kv: 
['p', 'Rey']
kv: 
['r', 'Louie']
cm:
[('p', 'Rey'), ('r', 'Louie')]
Rey is Zany. Louie is Zany. 
['Rey is Zany', 'Louie is Zany', '']


[('F', {'p'}), ('F', {'r'}), ('p', 'Rey'), ('r', 'Louie')]

In [11]:
parse_LLM_output(no_model_no_keys_but_correct)

kv: 
['F', 'Zany']
kv: 
['p', 'Rey']
kv: 
['r', 'Louie']
cm:
[('p', 'Rey'), ('r', 'Louie')]


[('F', {'p'}), ('F', {'r'}), ('p', 'Rey'), ('r', 'Louie')]

In [ ]:
# What else could happen?

In [20]:
re.split(r'keys', "World model: ... .keys: ... .keys: ... .", flags=re.IGNORECASE)

['World model: ... .', ': ... .', ': ... .']

In [10]:
re.split(r'keys', "World model: ... .", flags=re.IGNORECASE)

['World model: ... .']

In [5]:
re.split(r'keys', "World model: ... .keys: ... .", flags=re.IGNORECASE)

['World model: ... .', ': ... .']

In [15]:
a = re.split(r'keys', "", flags=re.IGNORECASE)
a

['']

In [48]:
b = " asdas "
b = b.strip()
b

'asdas'

In [58]:
[x for x in '/segment/segment/'.split('/') if x] # ignores the first empty string that is usually returned with split.

['segment', 'segment']

In [6]:
tmp = ': F: Conservative. q: Violet. l: George..'
tmp_list = re.split('\\.', tmp, flags=re.IGNORECASE)
reverse_predicate_mapping = {}
reverse_constant_mapping = {}
for k in tmp_list:
    if len(k) > 0: # splitting at the last "." leads to an empty string
        # clean the string (e.g. ":" after "keys:..." as well as white space after ":")
        k = k.lstrip("\\:")
        kv = k.split(":") # split keys and values
        kv = [i.lstrip() for i in kv] # clean whitespace
        print(kv)
        if kv[0].isupper(): #check for uppercase
            reverse_predicate_mapping[kv[1]] = kv[0] # e.g. Anxious: F. instead of: F: Anxious.
        else:
            reverse_constant_mapping[kv[1]] = kv[0]
print(reverse_constant_mapping)
print(reverse_predicate_mapping)

['F', 'Conservative']
['q', 'Violet']
['l', 'George']
{'Violet': 'q', 'George': 'l'}
{'Conservative': 'F'}
[('q', 'Violet'), ('l', 'George')]


In [15]:
reverse_constant_mapping = {"Rey":"a", "Louie":"b"}
reverse_predicate_mapping = {"Zany":"Z"}

In [17]:
nltk_expressions = []
    
world_model = "World model: Rey is Zany. Louie is Zany."
# clean the string
world_model = re.sub('world model', '', world_model, flags=re.IGNORECASE)
world_model = world_model.lstrip("\\:")
world_model = world_model.lstrip()
print(world_model)

world_model_list = re.split('\\.', world_model, flags=re.IGNORECASE)
world_model_list = [x.strip() for x in world_model_list if x]
print(world_model_list)

for sentence in world_model_list:
    # split sentence into words
    sentence = sentence.split()
    # get indices before and after is
    # here, we require that the LLM makes well-formed sentences (i.e. subject - verb "is" - [not] - adj). Otherwise, we do not take it).
    if "is" in sentence and sentence.index("is") > 0: 
        is_index = sentence.index("is")
        if "not" in sentence:
            if sentence.index("not") == is_index + 1: # again: needs to be grammatically correct
                try: # we may not have a reverse_predicate_mapping or reverse_constant_mapping, if keys are missing
                    name = sentence[is_index - 1] 
                    adjective = sentence[is_index + 2]
                    predicate = reverse_predicate_mapping[adjective] # mapping zum uppercase letter nötig!
                    constant = reverse_constant_mapping[name]
                    #nltk_expressions.append(NegatedExpression(ApplicationExpression(FunctionVariableExpression(Variable(predicate)), ConstantExpression(Variable(constant)))))
                    nltk_expressions.append((predicate, set(constant)))
                except:
                    pass
        else:
            try: 
                name = sentence[is_index - 1] 
                adjective = sentence[is_index + 1]
                predicate = reverse_predicate_mapping[adjective] # mapping zum uppercase letter nötig!
                constant = reverse_constant_mapping[name]
                #test = ApplicationExpression(FunctionVariableExpression(Variable(predicate)), ConstantExpression(Variable(constant)))
                nltk_expressions.append((predicate, set(constant)))
            except:
                pass
nltk_expressions

Rey is Zany. Louie is Zany.
['Rey is Zany', 'Louie is Zany']
Z
a
Z
b


[('Z', {'a'}), ('Z', {'b'})]

In [89]:
ex = "Rey is Zany"
ex = ex.split("is")
ex = [x.strip() for x in ex]
ex

['Rey', 'is', 'Zany']

In [96]:
ex = "Rey is Zany"
ex = ex.split()
print(ex)
if 'is' in ex:
    i = ex.index("is")
i

['Rey', 'is', 'Zany']


1

In [86]:
line = 'how are you?'

if len(line.split()) > 1:

False

In [14]:
accuracies = {}
accuracies_task1 = {}
accuracies_task1["a"] = 1
accuracies["task1"] = accuracies_task1
accuracies

{'task1': {'a': 1}}

### Test nltk parser

In [10]:
import nltk
from nltk.sem.logic import Expression 

read_expr = Expression.fromstring
formulas = ['exists xP(x)/asd'] * 100

In [ ]:
%%time
processed = []
# find the longest substring that can be parsed by the nltk parser. Has to include a "." after an exists or all quantifier
for formula in formulas:
    l = len(formula)
    start = formula.find('.')
    if start == -1:
        processed.append[""]
    else:
        successfully_parsed = []
        for i in range(start, l, 1):
            try:
                temp = read_expr(formula[0:i])
                successfully_parsed.append(temp)
            except:
                pass
        processed.append(successfully_parsed[-1])
        
# find the dot, do substring method after the dot! until the last (even) closing bracket that can be parsed.

In [ ]:
%%time
def postprocess_output(formula):
    l = len(formula)
    start = formula.find('.')
    successfully_parsed = []
    for i in range(start, l, 1):
        try:
            temp = read_expr(formula[0:i])
            successfully_parsed.append(temp)
        except:
            pass

processed2 = [postprocess_output(f) for f in formulas]

In [ ]:
%%time 
fin_list = list(map(postprocess_output, formulas))

In [ ]:
set(processed)